In [1]:
%autosave 60
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosaving every 60 seconds


## Auxiliary Functions

We need the parser for first order formulas, hence we import it.

In [2]:
import folParser as fp

def parse(s):
    return fp.LogicParser(s).parse()

The resolution calculus works with clauses.  The module <tt>folCNF</tt> implements the function $\texttt{normalize}(f)$ that turns a formula $f$ into a set of clauses.

In [3]:
import folCNF as cnf

The module <tt>unify</tt> implements unification.

In [4]:
import unify

In [5]:
def arb(S):
    for x in S:
        return x

Given a literal $l$ the function $\texttt{complement}(l)$ computes the complement $\overline{\,l\,}$ of the literal $l$.

In [6]:
def complement(l):
    "Compute the complement of the literal l."
    if l[0] == '¬':
        return l[1]
    else:
        return ('¬', l)

In [7]:
complement(('p', 'x'))

('¬', ('p', 'x'))

In [8]:
complement(('¬', ('p', 'x')))

('p', 'x')

Given a clause $C$, the function $\texttt{collectVariables}(C)$ computes the set of all variables occurring in $C$.  The function $\texttt{collectVariables}$ can also collect the variables in a literal or a term.

In [9]:
def collectVariables(C):
    if isinstance(C, frozenset):
        return { x for literal in C 
                   for x in collectVariables(literal) 
               }
    if isinstance(C, str): # C is a variable
        return { C }
    if C[0] == '¬':
        return collectVariables(C[1])
    # C must be a term or atomic formula
    args = C[1:]
    return { x for t in args for x in collectVariables(t) }

In [10]:
s = '∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c)))'
f = parse(s)
f

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [11]:
Clauses = cnf.normalize(f)
Clauses

{frozenset({('Parent', 'g', ('sk1', 'g', 'c')),
            ('¬', ('Grandparent', 'g', 'c'))}),
 frozenset({('Grandparent', 'g', 'c'),
            ('¬', ('Parent', 'a', 'c')),
            ('¬', ('Parent', 'g', 'a'))}),
 frozenset({('Parent', ('sk1', 'g', 'c'), 'c'),
            ('¬', ('Grandparent', 'g', 'c'))})}

In [12]:
for C in Clauses:
    print(f'collectVariables({C}) = \n\t{collectVariables(C)}')

collectVariables(frozenset({('Parent', 'g', ('sk1', 'g', 'c')), ('¬', ('Grandparent', 'g', 'c'))})) = 
	{'c', 'g'}
collectVariables(frozenset({('Grandparent', 'g', 'c'), ('¬', ('Parent', 'a', 'c')), ('¬', ('Parent', 'g', 'a'))})) = 
	{'c', 'a', 'g'}
collectVariables(frozenset({('Parent', ('sk1', 'g', 'c'), 'c'), ('¬', ('Grandparent', 'g', 'c'))})) = 
	{'c', 'g'}


In [13]:
import string

The function $\texttt{renameVariables}(f)$ renames the variables in $f$ so that they are different from $g$.

In [14]:
def renameVariables(f, g=frozenset()):
    Variables = collectVariables(f)
    NewVars   = set(string.ascii_lowercase) - Variables - collectVariables(g)
    NewVars   = sorted(list(NewVars))
    sigma     = { x: NewVars[i] for (i, x) in enumerate(Variables) }
    return unify.apply(f, sigma)

# A Calculus for First Order Logic

Given a two clauses <tt>C1</tt> and <tt>C2</tt>, the function $\texttt{resolve}(\texttt{C1}, \texttt{C2})$ computes a set of all clauses that can be inferred from <tt>C1</tt> and <tt>C2</tt> by applying the resolution rule.

In [15]:
def resolve(C1, C2):
    C2 = renameVariables(C2, C1)
    Result = set()
    for L1 in C1:
        for L2 in C2:
            mu = unify.unify(L1, complement(L2))
            if mu != None:
                C1C2 = unify.apply((C1 - { L1 }) | (C2 - { L2 }), mu)
                Result.add(C1C2)
    return Result

## Some Formulas for Testing

In [16]:
s1 = '∀x:(∀y:(Child(y, x) → CanFly(y)) → Happy(x))'
s2 = '∀x:(Red(x) → CanFly(x))'
s3 = '∀x:(Red(x) → ∀y:(Child(y, x) → Red(y)))'
s4 = '¬∀x:(Red(x) → Happy(x))'
f1 = parse(s1)
f2 = parse(s2)
f3 = parse(s3)
f4 = parse(s4)
Clauses = cnf.normalize(f1) | cnf.normalize(f2) | cnf.normalize(f3) | cnf.normalize(f4)
Clauses

{frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))}),
 frozenset({('¬', ('Happy', ('sk3',)))}),
 frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))}),
 frozenset({('Red', ('sk3',))}),
 frozenset({('Child', ('sk2', 'x'), 'x'), ('Happy', 'x')}),
 frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})}

In [17]:
C1 = frozenset({('Red', ('sk3',))})

In [18]:
C2 = frozenset({('¬', ('Happy', ('sk3',)))})

In [19]:
C3 = frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})

In [20]:
C4 = frozenset({('Child', ('sk2', 'x'), 'x'), ('Happy', 'x')})

In [21]:
C5 = frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))})

In [22]:
C6 = frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))})

In [23]:
C7 = arb(resolve(C1, C6))
C7

frozenset({('Red', 'b'), ('¬', ('Child', 'b', ('sk3',)))})

In [24]:
C8 = arb(resolve(C7, C4))
C8

frozenset({('Happy', ('sk3',)), ('Red', ('sk2', ('sk3',)))})

In [25]:
C9 = arb(resolve(C8, C2))
C9

frozenset({('Red', ('sk2', ('sk3',)))})

In [26]:
C10 = arb(resolve(C9, C3))
C10

frozenset({('CanFly', ('sk2', ('sk3',)))})

In [27]:
C11 = arb(resolve(C10, C5))
C11

frozenset({('Happy', ('sk3',))})

In [28]:
arb(resolve(C11, C2))

frozenset()

In [29]:
def factorize(C):
    Result = set()
    for L1 in C:
        for L2 in C:
            if L1 != L2:
                mu  = unify.unify(L1, L2)
                if mu != None:
                    Cmu = unify.apply(C, mu)
                    Result.add(Cmu)
    return Result

In [30]:
C7 = arb(cnf.normalize(parse('∀x:∀y:P(F(x),y) ∨ ∀u:∀v:P(u,G(v))')))
C7

frozenset({('P', 'u', ('G', 'v')), ('P', ('F', 'x'), 'y')})

In [31]:
factorize(C7)

{frozenset({('P', ('F', 'x'), ('G', 'v'))})}

In [32]:
C8 = arb(cnf.normalize(parse('∀x:∀y:(¬P(F(x),y)) ∨ ∀u:∀v:(¬P(u,G(v)))')))
C8

frozenset({('¬', ('P', ('F', 'x'), 'y')), ('¬', ('P', 'u', ('G', 'v')))})

In [33]:
factorize(C8)

{frozenset({('¬', ('P', ('F', 'x'), ('G', 'v')))})}

Given two sets of clauses <tt>Cs1</tt> and <tt>Cs2</tt>, the function $\texttt{infere}(\texttt{Cs1}, \texttt{Cs2})$ returns all possible clauses that result from:
<ol>
    <li>the resolution of a clause $C_1 \in \texttt{Cs1}$ with a clause $C_2 \in \texttt{Cs2}$,</li>
    <li>the resolution of two clauses $C_1, C_2 \in \texttt{Cs1}$,</li>
    <li>the factorization of a clause $C \in \texttt{Cs1}$.</li>
</ol>

In [34]:
def infere(Clauses):
    Result =  { C for C1 in Clauses
                  for C2 in Clauses
                  for C  in resolve(C1, C2)
              }
    Result |= { C for C1 in Clauses for C in factorize(C1) }
    return Result 

In [35]:
def prettyPrint(Clauses):
    for C in Clauses:
        print(set(C))

In [36]:
prettyPrint(Clauses)

{('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))}
{('¬', ('Happy', ('sk3',)))}
{('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x')), ('Red', 'y')}
{('Red', ('sk3',))}
{('Happy', 'x'), ('Child', ('sk2', 'x'), 'x')}
{('¬', ('Red', 'x')), ('CanFly', 'x')}


In [37]:
def saturate(Clauses):
    OldClauses = set()
    cnt        = 1
    while frozenset() not in Clauses:
        Clauses |= infere(Clauses)
        print(f'cnt = {cnt}, number of clauses: {len(Clauses)}')
        cnt += 1
    return True

In [38]:
saturate(Clauses)

cnt = 1, number of clauses: 19
cnt = 2, number of clauses: 171
cnt = 3, number of clauses: 19084


True